In [1]:
import polars as pl

In [ ]:
df = (
    pl.scan_csv(
        "../test/filtered_DMBD_BlastP.tsv.gz",
        separator="\t",
        has_header=False,
        new_columns=["query_gene", "target_gene", "identity", "bitscore"],
    )
    .with_columns(
        query_genome=pl.col("query_gene").str.replace(r"_\d+$", ""),
        target_genome=pl.col("target_gene").str.replace(r"_\d+$", ""),
    )
    .filter(
        pl.col("query_genome") != pl.col("target_genome"),
        pl.col("query_gene") != pl.col("target_gene"),
    )
    .group_by(
        pl.concat_arr(pl.col("query_genome"), pl.col("target_genome"))
        .arr.sort()
        .arr.to_struct(["genome_1", "genome_2"])
        .alias("genome_pair")
    )
    .agg(pl.len().alias("n_shared_genes"))
    .unnest("genome_pair")
    .collect(engine="streaming")
)

In [ ]:
df.head()

In [ ]:
df.write_csv("n_shared_genes_pairs.tsv", separator="\t")